In [2]:
import google.colab
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
import cv2

In [ ]:
features=[]
target=[]

In [ ]:
for x in["dark spots","puffy eyes","wrinkles","clear_face"]:
    ImagesNamesList=os.listdir("/content/drive/MyDrive/verzeo_ageing_dataset (1)" + "/" + str(x))
    for y in ImagesNamesList:
        Imgarr=cv2.imread("/content/drive/MyDrive/verzeo_ageing_dataset (1)" + "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(200,200))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="dark spots":
                target.append(0)
            elif x=="puffy eyes":
                target.append(1) 
            elif x=="wrinkles":
                target.append(2)
            else:
                target.append(3)
    print("In Folder", x)

In [ ]:
import numpy as np

In [ ]:
features=np.array(features)
target=np.array(target)

In [ ]:
features.shape

In [ ]:
target.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features_train,features_test,target_train,target_test=train_test_split(features,target,test_size=0.2)

In [ ]:
features_train.shape

In [ ]:
target_train.shape

In [ ]:
features_test.shape

In [ ]:
target_test.shape

In [ ]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255 
    return image

In [ ]:
features_train=np.array(list(map(preprocessing,features_train)))

In [ ]:
features_train.shape

In [ ]:
features_train=features_train.reshape((1192, 200, 200,1))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)

In [ ]:
dataGen.fit(features_train)

In [ ]:
batches=dataGen.flow(features_train,target_train,batch_size=20)

In [ ]:
images,labels=next(batches)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
for i in range(0,20):
    plt.subplot(4,5,i+1)
    plt.imshow(images[i].reshape(200,200))
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
target_train=to_categorical(target_train)

In [ ]:
target_train.shape

In [ ]:
features_train.shape

In [ ]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0


In [ ]:
conv_base=EfficientNetB0(weights="imagenet",include_top=False,input_shape=(244,244,3))
model=Sequential()
model.add(conv_base)
model.add(MaxPooling2D((3,3)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(4,activation="sigmoid"))

In [ ]:
from keras.optimizers import Adam

In [ ]:
model.compile(Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(dataGen.flow(features_train,target_train,batch_size=20),epochs=30)

In [ ]:
from keras.models import model_from_json

In [ ]:
model_json=model.to_json()
with open("signsOfAging.json","w") as abc:
    abc.write(model_json)
    abc.close()
model.save_weights("signsOfAging.h5")
print("Save the Model")

In [ ]:
json_file=open("signsOfAging.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("signsOfAging.h5")
print("Loaded Model Sucesssfully")

In [ ]:
 import numpy as np
import cv2

 

def getClassName(classNo):
    if   classNo == 0: return 'With dark spots'
    elif classNo == 1: return 'With puffy eyes'
    elif classno == 2: return 'With wrinkles '
    elif classno ==3:  return 'Glowing Skin !'

In [ ]:

image=cv2.imread("/content/03198defbf9905613bedbbdd2323228d.jpg")
imagearr=np.array(image)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
faces=face_cascade.detectMultiScale(gray,1.05,5)
for (x,y,w,h) in faces:
    detectedFace=image[x:x+w,y:y+h]
    detectedFace=cv2.resize(detectedFace,(244,244))
    detectedFace=preprocessing(detectedFace)
    detectedFace=detectedFace.reshape(1,200,200,1)
      predictions=loaded_model.predict(detectedFace)
    classIndex=loaded_model.predict_classes(detectedFace)
      probabilityValue=np.amax(predictions)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
      if probabilityValue>0.75:
        cv2.putText(image,getClassName(classIndex),(x , y-3),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
         cv2.putText(image,str(int(probabilityValue * 100)) + " %",( x+w+20,y-3),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2_imshow(image)
      returnedValue=cv2.waitKey(1)
    if returnedValue==ord("s") or returnedValue==ord("S"):
         cv2.destroyAllWindows()
        break